<a href="https://colab.research.google.com/github/amhaiskar0921/AmazonProject/blob/main/main_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# For data visualization
import seaborn as sns

In [ ]:
shopping_data = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_examples.parquet')
df = shopping_data.to_pandas()
shopping_data_p = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_products.parquet')
df_p = shopping_data_p.to_pandas()
df_p = dp_p.drop(columns=['product_locale']) #repeated column with df

In [4]:
df_merged = pd.merge(df, df_p, on='product_id', how='inner')
df_merged= df_merged.drop(["product_locale"])
print(f"number of products {len(pd.unique(df_merged['product_id']))}")
df_merged_no_null = df_merged.fillna("")
missing_values = df_merged_no_null.isnull().sum()
print(missing_values)
df_p.size

number of products 1802772
example_id              0
query                   0
query_id                0
product_id              0
product_locale_x        0
esci_label              0
small_version           0
large_version           0
split                   0
product_title           0
product_description     0
product_bullet_point    0
product_brand           0
product_color           0
product_locale_y        0
dtype: int64


12704468

In [5]:
# concat dat
!pip install transformers
import torch
import transformers
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.2 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Add this code before tokenization
def remove_stopwords(text):
    stop_words_english = set(stopwords.words('english'))
    stop_words_spanish = set(stopwords.words('spanish'))
    # stop_words_japanese = set(stopwords.words('japanese')) <-- resulted in error
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words_english
                      and word.lower() not in stop_words_spanish]
    return ' '.join(filtered_words)

In [8]:
# Get a random sample of 10000 elems in the merged dataset with null values replaced
sample_size = 10000
sampled_merged_no_null = df_merged_no_null.sample(n=sample_size, random_state=42)

# Initialize an empty list to store the modified input sequences
input_sequences = []

# Apply stopword removal to each element in the 'input_sequences' list using a for loop
for text in sampled_merged_no_null.apply(lambda x: f"[CLS] {x['product_title']} [SEP] {x['product_description']} [SEP] {x['product_bullet_point']} [SEP]", axis=1).tolist():
    cleaned_text = remove_stopwords(text)
    input_sequences.append(cleaned_text)


# print(input_sequences[:10])

In [ ]:
# input_sequences = sampled_merged_no_null.apply(lambda x: f"[CLS] {x['product_title']} [SEP] {x['product_description']} [SEP] {x['product_bullet_point']} [SEP]", axis=1).tolist()

In [9]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Example text to tokenize
# text = "This is an example sentence."

# Tokenize the text
tokens = tokenizer(input_sequences, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized text to PyTorch tensors
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [10]:
# input_data = tokenizer(sampled_merged_no_null, padding=True, truncation=True, return_tensors="pt", max_length=512)
# labels = torch.tensor(df["esci_label"].values.astype(int))

In [11]:
from torch.utils.data import DataLoader, TensorDataset
def get_data_loader(features, max_seq_length, batch_size, shuffle=True):
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader

In [12]:
class BertInputItem(object):
    """An item with all the necessary attributes for finetuning BERT."""

    def __init__(self, text, input_ids, input_mask, segment_ids, label_id):
        self.text = text
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


In [13]:
def convert_examples_to_inputs(example_texts, example_labels, label2idx, max_seq_length, tokenizer, verbose=0):
      """Loads a data file into a list of `InputBatch`s."""

      input_items = []
      examples = zip(example_texts, example_labels)
      for (ex_index, (text, label)) in enumerate(examples):

          # Create a list of token ids
          input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
          if len(input_ids) > max_seq_length:
              input_ids = input_ids[:max_seq_length]

          # All our tokens are in the first input segment (id 0).
          segment_ids = [0] * len(input_ids)

          # The mask has 1 for real tokens and 0 for padding tokens. Only real
          # tokens are attended to.
          input_mask = [1] * len(input_ids)

          # Zero-pad up to the sequence length.
          padding = [0] * (max_seq_length - len(input_ids))
          input_ids += padding
          input_mask += padding
          segment_ids += padding

          assert len(input_ids) == max_seq_length
          assert len(input_mask) == max_seq_length
          assert len(segment_ids) == max_seq_length

          label_id = label2idx[label]

          input_items.append(
              BertInputItem(text=text,
                            input_ids=input_ids,
                            input_mask=input_mask,
                            segment_ids=segment_ids,
                            label_id=label_id))


      return input_items

In [14]:
rest_texts, test_texts, rest_labels, test_labels = train_test_split(input_sequences, sampled_merged_no_null['esci_label'].tolist(), test_size=0.1, random_state=1)
train_texts, dev_texts, train_labels, dev_labels = train_test_split(rest_texts, rest_labels, test_size=0.1, random_state=1)

In [15]:
target_names = list(set(sampled_merged_no_null['esci_label'].tolist()))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)

{'S': 0, 'I': 1, 'E': 2, 'C': 3}


In [ ]:
# inputItem = BertInputItem()
MAX_SEQ_LENGTH = 687 # No idea what this means :(
train_features = convert_examples_to_inputs(train_texts, train_labels, label2idx, MAX_SEQ_LENGTH, tokenizer, verbose=0)
dev_features = convert_examples_to_inputs(dev_texts, dev_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_inputs(test_texts, test_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)

BATCH_SIZE = 16

train_dataloader = get_data_loader(train_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=True)
dev_dataloader = get_data_loader(dev_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)
test_dataloader = get_data_loader(test_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)

In [21]:
sampled_merged_no_null.head()

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
1702727,1328975,mens 7.5 wide black loafers,67412,B07J48HG89,us,E,0,1,train,"Skechers Men's Equalizer 3.0 Bluegate Loafer, ...",,Breathable mesh upper\nAir coold memory foam i...,Skechers,Black/Grey,us
709385,382533,bookcase storage with doors,18390,B01M12PC36,us,E,0,1,train,"C&AHOME Cube Storage Organizer with Doors, 6-C...","<b>C&AHOME Cube Storage Organizer with Doors, ...",ENOUGH STORAGE SPACE : Can store all sort of i...,C&AHOME,White With Crack Black Color,us
1729484,1360462,miles davis cd,69073,B003O5MOFW,us,E,0,1,train,"""Four"" & More",,Miles Davis- Four & More,Legacy,,us
1868642,1536091,patrones punto de cruz,78167,B07G175JJM,es,E,1,1,train,Decdeal Punto de Cruz Patrón de Pareja de Anci...,"<b>caracteristicas:</b><br /> Punto de cruz ""T...","Punto de cruz ""The Old Married Couple"", hermos...",Decdeal,,es
1145286,733502,easy rider blu ray,36510,B00579EL86,us,E,0,1,train,Films cultes - Coffret - Easy Rider + Taxi Dri...,Quick Shipping !!! New And Sealed !!! This Dis...,,,,us
